# Extracting the data b/w neural activity and images

In [1]:
! pip install --upgrade caveclient

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for caveclient: filename=caveclient-5.15.2-py3-none-any.whl size=75810 sha256=8098d81db2b230078790bb3b2816b7061b2aca1c953dfea075f049802a061ce3
  Stored in directory: /home/vscode/.cache/pip/wheels/69/fc/fc/75346c997650659945c7828e277ef5015575b23f5d5a7133be
Successfully built caveclient
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
microns-phase3 0.8.0 requires caveclient==5.7.0, but you have caveclient 5.15.2 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import datajoint as dj
from caveclient import CAVEclient
from microns_phase3 import nda, utils
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`
Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


In [2]:
client = CAVEclient()
client.auth.save_token("64ea0271652fd86dc7fb6376cd9d52b9")

In [3]:
client = CAVEclient('minnie65_public')
client.materialize.version = 661

## Data Creation

In [4]:
in_ex_ba = pd.read_pickle('./in_ex_neurons_ba.pkl')

In [5]:
in_ex_ba.head()

,id,session,scan_idx,unit_id,pt_root_id,pt_position,brain_area
0,13434,7.0,5.0,4909.0,864691135738685297,"[338720, 125232, 19589]",AL
1,3045,7.0,4.0,9575.0,864691135614842827,"[136400, 170640, 17951]",V1
2,1474,9.0,4.0,8066.0,864691135502985397,"[189472, 119296, 26037]",V1
3,7915,6.0,7.0,6413.0,864691135387371905,"[209328, 174304, 20004]",V1
4,11589,9.0,3.0,7748.0,864691135661410544,"[185088, 157776, 15691]",V1


In [6]:
in_ex_ba.shape

(56312, 7)

In [7]:
root_ba_dict = dict(zip(in_ex_ba['pt_root_id'], in_ex_ba['brain_area']))
root_id_dict = dict(zip(in_ex_ba['pt_root_id'], in_ex_ba['id']))
root_session_dict =  dict(zip(in_ex_ba['pt_root_id'], in_ex_ba['session']))
root_scan_dict =  dict(zip(in_ex_ba['pt_root_id'], in_ex_ba['scan_idx']))
root_unit_dict =  dict(zip(in_ex_ba['pt_root_id'], in_ex_ba['unit_id']))
root_position_dict =  dict(zip(in_ex_ba['pt_root_id'], in_ex_ba['pt_position']))

In [48]:
def connectome_constructor(client, presynaptic_set, postsynaptic_set, neurs_per_steps = 500):
    '''
    Function to construct the connectome subset for the neurons specified in the presynaptic_set and postsynaptic_set.

    Args:
    client: CAVEclient needed to access MICrONS connectomics data
    presynaptic_set: 1-d array of non repeated root_ids of presynaptic neurons for which to extract postsynaptoc connections in postynaptic_set
    postynaptic_set: 1-d array of non repeated root_ids of postsynaptic neurons for which to extract presynaptic connections in presynaptic_set
    neurs_per_steps: number of postsynaptic neurons for which to recover presynaptic connectivity per single call to the connectomics
        database. Since the connectomics database has a limit on the number of connections you can query at once
        this iterative method optimises querying multiple neurons at once, as opposed to each single neuron individually,
        while also preventing the queries from crashing. I have tested that for a presynaptic set of around 8000 neurons
        you can reliably extract the connectivity for around 500 postsynaptic neurons at a time.
    '''
    
    if_thresh = (postsynaptic_set.shape[0]//neurs_per_steps)*neurs_per_steps
    
    syndfs = []
    for i in tqdm(range(0, postsynaptic_set.shape[0], neurs_per_steps), desc = 'Extracting connectome subset'):
        
        if i <if_thresh:
            post_ids = postsynaptic_set[i:i+neurs_per_steps]

        else:
            post_ids = postsynaptic_set[i:]

        sub_syn_df = client.materialize.query_table('synapses_pni_2',
                                            filter_in_dict={'pre_pt_root_id': presynaptic_set,
                                                            'post_pt_root_id':post_ids})
            
        syndfs.append(np.array(sub_syn_df[['pre_pt_root_id', 'post_pt_root_id', 'size']]))
    
    syn_df = pd.DataFrame({'pre_pt_root_id':np.vstack(syndfs)[:, 0], 'post_pt_root_id': np.vstack(syndfs)[:, 1], 'size': np.vstack(syndfs)[:, 2]})
    return syn_df
    


In [ ]:
def connectome_constructor(postsynaptic_set, neurons_per_step=500):
    synfds = []
    if_thresh = (postsynaptic_set.shape[0]//neurons_per_step)*neurons_per_step
    for i in tqdm(range(0, postsynaptic_set.shape[0], neurons_per_step)):
        if i <if_thresh:
            post_ids = postsynaptic_set[i:i+neurons_per_step]

        else:
            post_ids = postsynaptic_set[i:]
    
    sub_syn_df = client.materialize.synapse_query(post_ids=post_ids)
    return sub_syn_df


In [9]:
ids = in_ex_ba['pt_root_id']

In [13]:
syn_df1 = connectome_constructor(ids[:100], 500)

Exception ignored in: <function tqdm.__del__ at 0x7fd22508c0d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/usr/local/lib/python3.9/site-packages/tqdm/notebook.py", line 277, in close
    if self.disable:
AttributeError: 'tqdm' object has no attribute 'disable'


TraitError: The 'value' trait of a HTML instance expected a unicode string, not the int 500.

In [59]:
syn_df1

,pre_pt_root_id,post_pt_root_id,size
0,864691136991031445,864691136991031445,268
1,864691136011607715,864691136011607715,260
2,864691135938356484,864691135938356484,30724
3,864691135938356484,864691135938356484,1496
4,864691135938356484,864691135938356484,896
...,...,...,...
8581,864691136021058296,864691135783221043,280
8582,864691136673575047,864691136287463619,3164
8583,864691135565618007,864691135518280074,1084
8584,864691135272867601,864691135489264442,1704


In [10]:
syn_df2 = connectome_constructor(client, ids[8000:16000], ids[8000:16000], 500)

In [11]:
syn_df3 = connectome_constructor(client, ids[16000:24000], ids[16000:24000], 500)

In [12]:
syn_df4 = connectome_constructor(client, ids[24000:32000], ids[24000:32000], 500)

In [13]:
syn_df5 = connectome_constructor(client, ids[32000:40000], ids[32000:40000], 500)

In [15]:
syn_df6 = connectome_constructor(client, ids[48000:56312], ids[48000:56312], 500)

In [16]:
syn_df = pd.concat([syn_df1, syn_df2, syn_df3, syn_df4, syn_df5, syn_df6])

In [ ]:
syn_df.to_pickle('./in_ex_syn.pkl')

## Data manipulation

In [10]:
syn_df = pd.read_pickle('./in_ex_syn.pkl')

In [11]:
syn_df.head()

,pre_pt_root_id,post_pt_root_id
0,864691135560492001,864691135560492001
1,864691135560492001,864691135560492001
2,864691135910899113,864691135910899113
3,864691135661221360,864691135661221360
4,864691136388640119,864691136481342436


In [20]:
syn_df['pre_brain_area'] = syn_df['pre_pt_root_id'].map(root_ba_dict)
syn_df['post_brain_area'] = syn_df['post_pt_root_id'].map(root_ba_dict)

In [27]:
root_ba_dict

{864691135738685297: 'AL',
 864691135614842827: 'V1',
 864691135502985397: 'V1',
 864691135387371905: 'V1',
 864691135661410544: 'V1',
 864691135349275351: 'RL',
 864691137197706305: 'V1',
 864691136602371025: 'RL',
 864691135065322052: 'V1',
 864691135731072697: 'V1',
 864691135565275479: 'V1',
 864691136194984268: 'V1',
 864691135938356484: 'V1',
 864691135975395439: 'V1',
 864691136923754212: 'V1',
 864691136619416973: 'V1',
 864691136673575047: 'V1',
 864691135544785448: 'RL',
 864691135272867601: 'V1',
 864691135577813765: 'V1',
 864691136309871706: 'RL',
 864691135737772676: 'RL',
 864691135866987798: 'V1',
 864691135759371086: 'RL',
 864691135013895574: 'V1',
 864691136145003159: 'V1',
 864691135407816393: 'V1',
 864691136119541016: 'V1',
 864691135463695429: 'RL',
 864691135841932515: 'RL',
 864691135725228203: 'V1',
 864691135133547168: 'V1',
 864691137020483694: 'V1',
 864691136379765205: 'RL',
 864691135233954137: 'V1',
 864691136324904087: 'V1',
 864691135195405354: 'V1',
 

In [28]:
864691135738685297 == 864691135738685297

True

In [19]:
syn_df[syn_df['pre_pt_root_id'] == 864691135738685297]

,pre_pt_root_id,post_pt_root_id,pre_brain_area_,post_brain_area
3211,864691135738685297,864691135738685297,AL,AL
5990,864691135738685297,864691135738685297,AL,AL
5994,864691135738685297,864691135738685297,AL,AL
7286,864691135738685297,864691135738685297,AL,AL
7300,864691135738685297,864691135738685297,AL,AL
7301,864691135738685297,864691135738685297,AL,AL
7303,864691135738685297,864691135738685297,AL,AL
7305,864691135738685297,864691135738685297,AL,AL
7308,864691135738685297,864691135738685297,AL,AL
7326,864691135738685297,864691135738685297,AL,AL


## Continuare dopo

In [ ]:
unit_keys = in_ex_ba[['session', 'scan_idx', 'unit_id']].to_dict(orient='records')

In [ ]:
nframes, fps = (nda.Scan & unit_keys[0]).fetch1('nframes', 'fps')  # fetch # frames and fps
time_axis = np.arange(nframes)/ fps # create time axis (seconds)
spike_trace = (nda.Activity & unit_keys).fetch1('trace') # fetch spike trace
calcium_trace = (nda.ScanUnit * nda.Fluorescence & unit_keys).fetch1('trace') # fetch calcium fluorescence trace
pupil_radius = (nda.ManualPupil & unit_keys).fetch1('pupil_maj_r') # fetch manually segmented pupil trace 

In [ ]:
nframes, fps = (nda.Scan & unit_keys).fetch('nframes', 'fps')  # fetch # frames and fps

In [ ]:
nframes_list = []
fps_list = []
for k in unit_keys:
    nframes, fps = (nda.Scan & k).fetch1('nframes', 'fps')
    nframes_list.append(nframes)
    fps_list.append(nframes)

In [ ]:
len(nframes_list)v